## import libraries

In [1]:
import logging
import os
from pathlib import Path
from typing import Any, List

from tqdm import tqdm
import pandas as pd
import numpy as np
import hydra
import lightning as L
import lightning.pytorch as pl
import peft
import torch
from datasets import DatasetDict, load_dataset
from hydra.utils import instantiate
from omegaconf import DictConfig, ListConfig, OmegaConf
from peft.tuners.lora import LoraLayer
from torch import Tensor, nn
from torch.utils.data import DataLoader
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer, default_data_collator

import peta
from peta.models.LinearizedModel import LinearizedModelWraper
from peta.utils import TimeIt, TitledLog
from peta.utils.logging.rich import pprint_yaml, setup_colorlogging
from peta.utils.ml.devices import num_devices

log = logging.getLogger(__name__)

# disable tokenizers parallelism
os.environ["TOKENIZERS_PARALLELISM"] = "false"

from finetune_lm import load_model_from_config

In [2]:
fabric = L.Fabric(accelerator="cuda", devices=[0])
fabric.launch()

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


In [3]:
def load_pretrained_model(
    model: str,
    dataset: str,
    finetune_mode: str,
    version: int,
):
    log_dir: Path = (
        Path("logs") / model / dataset / finetune_mode / f"version_{version}"
    )
    config_path = log_dir / "config.yaml"
    cfg: DictConfig = OmegaConf.load(config_path)

    # load model from config
    with TitledLog("load pretrained model and tokenizer", log_fn=log.info):
        _return = load_model_from_config(cfg)
        tokenizer: AutoTokenizer = _return["tokenizer"]
        model: AutoModelForSeq2SeqLM | peft.PeftModel = _return["model"]
        model.eval()

    return {
        "config": cfg,
        "model": model,
        "tokenizer": tokenizer,
    }

In [4]:
def load_finetuned_model(
    model: str,
    dataset: str,
    finetune_mode: str,
    version: int,
):
    log_dir: Path = (
        Path("logs") / model / dataset / finetune_mode / f"version_{version}"
    )
    config_path = log_dir / "config.yaml"
    cfg: DictConfig = OmegaConf.load(config_path)

    # load model from config
    with TitledLog("load pretrained model and tokenizer", log_fn=log.info):
        _return = load_model_from_config(cfg)
        tokenizer: AutoTokenizer = _return["tokenizer"]
        model: AutoModelForSeq2SeqLM | peft.PeftModel = _return["model"]

        # load checkpoint
        checkpoint_dir = log_dir / "checkpoints"
        checkpoints = os.listdir(checkpoint_dir)
        # get checkpoint files with `step=2000.pth` in its name
        checkpoints = list(filter(lambda x: "step=2000.pth" in x, checkpoints))
        # assert (
        #     len(checkpoints) == 1
        # ), f"multiple checkpoints found, found checkpoints: {checkpoints}, checkpoint dir: {checkpoint_dir}"
        assert (
            len(checkpoints) >= 1
        ), f"no checkpoint found, checkpoint dir: {checkpoint_dir}"
        if len(checkpoints) > 1:
            log.warn(
                f"multiple checkpoints found, found checkpoints: {checkpoints}, checkpoint dir: {checkpoint_dir}"
            )
        checkpoint = checkpoints[0]
        log.info(f"load checkpoint from {checkpoint}")

        # load trainable parameters
        state_dict = torch.load(checkpoint_dir / checkpoint, map_location="cpu")
        model.load_state_dict(state_dict, strict=False)
        model.eval()

    return {
        "config": cfg,
        "model": model,
        "tokenizer": tokenizer,
    }

In [24]:
model = "flan-t5-base"
dataset = "glue-cola"
finetune_mode = "standard"

model = load_finetuned_model(model, dataset, finetune_mode, 0)
cfg, model, tokenizer = model["config"], model["model"], model["tokenizer"]

datasets = instantiate(cfg.dataset.datasets)

if "validation" in datasets:
    val_dataset = datasets["validation"]
elif "validation_matched" in datasets:
    # mnli
    val_dataset = datasets["validataion_matched"]
else:
    raise KeyError(datasets.keys())

# convert the task to text-to-text format
if hasattr(cfg.dataset, "preprocessor"):
    preprocessor = instantiate(
        cfg.dataset.preprocessor,
        tokenizer=tokenizer,
        tokenizer_kwargs=cfg.model.tokenizer_kwargs
        if hasattr(cfg.model, "tokenizer_kwargs")
        else None,
    )
    val_dataset = val_dataset.map(
        preprocessor,
        **cfg.dataset.map_kwargs if hasattr(cfg.dataset, "map_kwargs") else {},
    )

val_loader = DataLoader(
    val_dataset,
    batch_size=8,
    num_workers=0,
    shuffle=False,
    collate_fn=default_data_collator,
)

[15:39:00] INFO     =======load pretrained model and tokenizer========                              ]8;id=825329;file:///tmp/ipykernel_1323327/501775352.py\501775352.py]8;;\:]8;id=581389;file:///tmp/ipykernel_1323327/501775352.py#14\14]8;;\

[15:39:03] INFO     no peft config found, use full finetuning.                                   ]8;id=538520;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py\finetune_lm.py]8;;\:]8;id=932733;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py#128\128]8;;\

           INFO     load checkpoint from epoch=3_step=2000.pth                                      ]8;id=890846;file:///tmp/ipykernel_1323327/501775352.py\501775352.py]8;;\:]8;id=21436;file:///tmp/ipykernel_1323327/501775352.py#28\28]8;;\

           INFO     ==================================================                              ]8;id=322444;file:///tmp/ipykernel_1323327/501775352.py\501775352.py]8;;\:]8;id=371554;file:///tmp/ipykernel_1323327/501775352.py#14\14]8;;\

In [9]:
model = fabric.setup_module(model)
val_loader = fabric.setup_dataloaders(val_loader)

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


In [5]:
def remove_special_tokens(tokenizer, token_list):
    ret = []
    for token in token_list:
        if token not in tokenizer.all_special_ids and token > 0:
            ret.append(token)
        if token == -100:
            break
    return ret

In [6]:
def evaluate_accuracy(model, val_loader: DataLoader, tokenizer):
    from tqdm import tqdm
    correct = 0
    total = 0

    model = model.eval()
    for batch_idx, batch in enumerate(tqdm(val_loader)):
        with torch.no_grad():
            outputs = model.generate(batch["input_ids"])
            output_text = tokenizer.batch_decode(outputs, skip_special_tokens=True)

            labels = [remove_special_tokens(tokenizer, l) for l in batch["labels"]]
            labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

            # compare output_text and labels
            for i, j in zip(output_text, labels):
                if i == j:
                    correct += 1
                total += 1

    # return accuracy
    return correct / total

In [12]:
evaluate_accuracy(model, val_loader, tokenizer=tokenizer)

  0%|          | 0/131 [00:00<?, ?it/s]/data/users/tanganke/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 131/131 [00:19<00:00,  6.62it/s]


0.6951102588686481

# ZeroShot Performance

In [7]:
def load_pretrained_model_and_dataset(
    model: str, dataset: str, finetune_mode: str = "standard", version: int = 0
):
    model = load_pretrained_model(
        model,
        dataset,
        finetune_mode=finetune_mode,
        version=version,
    )
    cfg, model, tokenizer = model["config"], model["model"], model["tokenizer"]

    datasets = instantiate(cfg.dataset.datasets)

    if "validation" in datasets:
        val_dataset = datasets["validation"]
    elif "validation_matched" in datasets:
        # mnli
        val_dataset = datasets["validation_matched"]
    else:
        raise KeyError(datasets.keys())

    # convert the task to text-to-text format
    if hasattr(cfg.dataset, "preprocessor"):
        preprocessor = instantiate(
            cfg.dataset.preprocessor,
            tokenizer=tokenizer,
            tokenizer_kwargs=cfg.model.tokenizer_kwargs
            if hasattr(cfg.model, "tokenizer_kwargs")
            else None,
        )
        val_dataset = val_dataset.map(
            preprocessor,
            **cfg.dataset.map_kwargs if hasattr(cfg.dataset, "map_kwargs") else {},
        )

    val_loader = DataLoader(
        val_dataset,
        batch_size=32,
        num_workers=0,
        shuffle=False,
        collate_fn=default_data_collator,
    )

    model = fabric.setup_module(model)
    val_loader = fabric.setup_dataloaders(val_loader)

    return model, tokenizer, val_loader

In [26]:
zeroshot_results = {"model": [], "dataset": [], "accuracy": []}

model_name = "flan-t5-base"
for dataset_name in [
    "glue-cola",
    "glue-mnli",
    "glue-mrpc",
    "glue-qqp",
    "glue-rte",
    "glue-sst2",
    "glue-stsb",
]:
    model, tokenizer, val_loader = load_pretrained_model_and_dataset(
        model_name, dataset_name, "standard", 0
    )
    acc = evaluate_accuracy(model, val_loader, tokenizer)

    zeroshot_results["model"].append(model_name)
    zeroshot_results["dataset"].append(dataset_name)
    zeroshot_results["accuracy"].append(acc)

zeroshot_results = pd.DataFrame(zeroshot_results)
zeroshot_results

[15:39:17] INFO     =======load pretrained model and tokenizer========                             ]8;id=683304;file:///tmp/ipykernel_1323327/2329344656.py\2329344656.py]8;;\:]8;id=623505;file:///tmp/ipykernel_1323327/2329344656.py#14\14]8;;\

[15:39:20] INFO     no peft config found, use full finetuning.                                   ]8;id=411825;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py\finetune_lm.py]8;;\:]8;id=234863;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py#128\128]8;;\

           INFO     ==================================================                             ]8;id=124672;file:///tmp/ipykernel_1323327/2329344656.py\2329344656.py]8;;\:]8;id=335831;file:///tmp/ipykernel_1323327/2329344656.py#14\14]8;;\

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
100%|██████████| 33/33 [00:25<00:00,  1.29it/s]


[15:39:49] INFO     =======load pretrained model and tokenizer========                             ]8;id=987109;file:///tmp/ipykernel_1323327/2329344656.py\2329344656.py]8;;\:]8;id=571087;file:///tmp/ipykernel_1323327/2329344656.py#14\14]8;;\

...

[16:05:22] INFO     no peft config found, use full finetuning.                                   ]8;id=478522;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py\finetune_lm.py]8;;\:]8;id=532869;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py#128\128]8;;\

           INFO     ==================================================                             ]8;id=745175;file:///tmp/ipykernel_1323327/2329344656.py\2329344656.py]8;;\:]8;id=836771;file:///tmp/ipykernel_1323327/2329344656.py#14\14]8;;\

Running tokenizer on dataset:   0%|          | 0/1500 [00:00<?, ? examples/s]

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
100%|██████████| 47/47 [00:38<00:00,  1.22it/s]


,model,dataset,accuracy
0,flan-t5-base,glue-cola,0.0
1,flan-t5-base,glue-mnli,0.0
2,flan-t5-base,glue-mrpc,0.0
3,flan-t5-base,glue-qqp,0.0
4,flan-t5-base,glue-rte,0.0
5,flan-t5-base,glue-sst2,0.0
6,flan-t5-base,glue-stsb,0.0


# Full-Finetuned

In [8]:
def load_validation_dataloaer(cfg: DictConfig, batch_size=32):
    datasets = instantiate(cfg.dataset.datasets)

    if "validation" in datasets:
        val_dataset = datasets["validation"]
    elif "validation_matched" in datasets:
        # mnli
        val_dataset = datasets["validation_matched"]
    else:
        raise KeyError(datasets.keys())

    # convert the task to text-to-text format
    if hasattr(cfg.dataset, "preprocessor"):
        preprocessor = instantiate(
            cfg.dataset.preprocessor,
            tokenizer=tokenizer,
            tokenizer_kwargs=cfg.model.tokenizer_kwargs
            if hasattr(cfg.model, "tokenizer_kwargs")
            else None,
        )
        val_dataset = val_dataset.map(
            preprocessor,
            **cfg.dataset.map_kwargs if hasattr(cfg.dataset, "map_kwargs") else {},
        )

    val_loader = DataLoader(
        val_dataset,
        batch_size=batch_size,
        num_workers=0,
        shuffle=False,
        collate_fn=default_data_collator,
    )

    return val_loader

In [9]:
validataion_dataloaders = {}

In [12]:
fullfinetuned_results = {"model": [], "dataset": [], "accuracy": [], "config": []}

model_name = "flan-t5-base"
finetune_mode = "standard"
for dataset_name in [
    "glue-cola",
    "glue-mnli",
    "glue-mrpc",
    "glue-qqp",
    "glue-rte",
    "glue-sst2",
    "glue-stsb",
]:
    model = load_finetuned_model(model_name, dataset_name, finetune_mode, 0)
    cfg, model, tokenizer = model["config"], model["model"], model["tokenizer"]

    if dataset_name in validataion_dataloaders:
        val_loader = validataion_dataloaders[dataset_name]
    else:
        val_loader = load_validation_dataloaer(cfg)
        validataion_dataloaders[dataset_name] = val_loader

    model = fabric.setup_module(model)
    val_loader = fabric.setup_dataloaders(val_loader)
    acc = evaluate_accuracy(model, val_loader, tokenizer)

    fullfinetuned_results["model"].append(model_name)
    fullfinetuned_results["dataset"].append(dataset_name)
    fullfinetuned_results["accuracy"].append(acc)
    fullfinetuned_results["config"].append(cfg)

    print("model: {}, dataset: {}, accuracy: {}".format(model_name, dataset_name, acc))

fullfinetuned_results = pd.DataFrame(fullfinetuned_results)
fullfinetuned_results

[18:17:08] INFO     =======load pretrained model and tokenizer========                              ]8;id=414858;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=550654;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

[18:17:12] INFO     no peft config found, use full finetuning.                                   ]8;id=341932;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py\finetune_lm.py]8;;\:]8;id=701077;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py#128\128]8;;\

           INFO     load checkpoint from epoch=3_step=2000.pth                                      ]8;id=647399;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=848280;file:///tmp/ipykernel_1757684/993648505.py#35\35]8;;\

           INFO     ==================================================                              ]8;id=92983;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=632879;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
  0%|          | 0/33 [00:00<?, ?it/s]/data/users/tanganke/anaconda3/lib/python3.10/site-packages/transformers/generation/utils.py:1369: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
100%|██████████| 33/33 [06:00<00:00, 10.94s/it]

...

           INFO     load checkpoint from epoch=5_step=2000.pth                                      ]8;id=577984;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=569707;file:///tmp/ipykernel_1757684/993648505.py#35\35]8;;\

[18:55:51] INFO     ==================================================                              ]8;id=581784;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=287369;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
100%|██████████| 47/47 [00:28<00:00,  1.64it/s]

model: flan-t5-base, dataset: glue-stsb, accuracy: 0.196


,model,dataset,accuracy,config
0,flan-t5-base,glue-cola,0.695110,"[model, peft, dataset, optim, seed, batch_size..."
1,flan-t5-base,glue-mnli,0.814162,"[model, peft, dataset, optim, seed, batch_size..."
2,flan-t5-base,glue-mrpc,0.823529,"[model, peft, dataset, optim, seed, batch_size..."
3,flan-t5-base,glue-qqp,0.829335,"[model, peft, dataset, optim, seed, batch_size..."
4,flan-t5-base,glue-rte,0.851986,"[model, peft, dataset, optim, seed, batch_size..."
5,flan-t5-base,glue-sst2,0.933486,"[model, peft, dataset, optim, seed, batch_size..."
6,flan-t5-base,glue-stsb,0.196000,"[model, peft, dataset, optim, seed, batch_size..."


In [13]:
os.makedirs(f"results/{model_name}", exist_ok=True)
fullfinetuned_results.to_csv(f"results/{model_name}/fullfinetuned_results.csv")

## V1

In [22]:
fullfinetuned_results = {"model": [], "dataset": [], "accuracy": [], "config": []}

model_name = "flan-t5-base"
finetune_mode = "standard"
for dataset_name in [
    "glue-cola",
    "glue-mnli",
    "glue-mrpc",
    "glue-qqp",
    "glue-rte",
    "glue-sst2",
    "glue-stsb",
]:
    model = load_finetuned_model(model_name, dataset_name, finetune_mode, 1) # V1
    cfg, model, tokenizer = model["config"], model["model"], model["tokenizer"]

    if dataset_name in validataion_dataloaders:
        val_loader = validataion_dataloaders[dataset_name]
    else:
        val_loader = load_validation_dataloaer(cfg)
        validataion_dataloaders[dataset_name] = val_loader

    model = fabric.setup_module(model)
    val_loader = fabric.setup_dataloaders(val_loader)
    acc = evaluate_accuracy(model, val_loader, tokenizer)

    fullfinetuned_results["model"].append(model_name)
    fullfinetuned_results["dataset"].append(dataset_name)
    fullfinetuned_results["accuracy"].append(acc)
    fullfinetuned_results["config"].append(cfg)

    print("model: {}, dataset: {}, accuracy: {}".format(model_name, dataset_name, acc))

fullfinetuned_results = pd.DataFrame(fullfinetuned_results)
fullfinetuned_results

[21:36:24] INFO     =======load pretrained model and tokenizer========                              ]8;id=529903;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=631262;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

[21:36:27] INFO     no peft config found, use full finetuning.                                   ]8;id=681453;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py\finetune_lm.py]8;;\:]8;id=735392;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py#128\128]8;;\

/tmp/ipykernel_1757684/993648505.py:31: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  log.warn(


           WARNING  multiple checkpoints found, found checkpoints: ['epoch=4_step=2000.pth',        ]8;id=617889;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=291704;file:///tmp/ipykernel_1757684/993648505.py#31\31]8;;\
                    'epoch=3_step=2000.pth'], checkpoint dir:                                                      
                    logs/flan-t5-base/glue-cola/standard/version_1/checkpoints                                     

           INFO     load checkpoint from epoch=4_step=2000.pth                                      ]8;id=844962;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=167414;file:///tmp/ipykernel_1757684/993648505.py#35\35]8;;\

...

           INFO     load checkpoint from epoch=5_step=2000.pth                                      ]8;id=524902;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=798975;file:///tmp/ipykernel_1757684/993648505.py#35\35]8;;\

[21:56:18] INFO     ==================================================                              ]8;id=912804;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=655674;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
100%|██████████| 47/47 [00:27<00:00,  1.73it/s]

model: flan-t5-base, dataset: glue-stsb, accuracy: 0.19866666666666666


,model,dataset,accuracy,config
0,flan-t5-base,glue-cola,0.752637,"[model, peft, dataset, optim, seed, batch_size..."
1,flan-t5-base,glue-mnli,0.824147,"[model, peft, dataset, optim, seed, batch_size..."
2,flan-t5-base,glue-mrpc,0.857843,"[model, peft, dataset, optim, seed, batch_size..."
3,flan-t5-base,glue-qqp,0.836087,"[model, peft, dataset, optim, seed, batch_size..."
4,flan-t5-base,glue-rte,0.851986,"[model, peft, dataset, optim, seed, batch_size..."
5,flan-t5-base,glue-sst2,0.933486,"[model, peft, dataset, optim, seed, batch_size..."
6,flan-t5-base,glue-stsb,0.198667,"[model, peft, dataset, optim, seed, batch_size..."


In [23]:
os.makedirs(f"results/{model_name}", exist_ok=True)
fullfinetuned_results.to_csv(f"results/{model_name}/fullfinetuned_results_v1.csv")

# LoRA

## V0

In [14]:
lora_results = {"model": [], "dataset": [], "accuracy": [], "config": []}

model_name = "flan-t5-base"
finetune_mode = "lora"
for dataset_name in [
    "glue-cola",
    "glue-mnli",
    "glue-mrpc",
    "glue-qqp",
    "glue-rte",
    "glue-sst2",
    "glue-stsb",
]:
    model = load_finetuned_model(model_name, dataset_name, finetune_mode, 0)
    cfg, model, tokenizer = model["config"], model["model"], model["tokenizer"]

    if dataset_name in validataion_dataloaders:
        val_loader = validataion_dataloaders[dataset_name]
    else:
        val_loader = load_validation_dataloaer(cfg)
        validataion_dataloaders[dataset_name] = val_loader

    model = fabric.setup_module(model)
    val_loader = fabric.setup_dataloaders(val_loader)
    acc = evaluate_accuracy(model, val_loader, tokenizer)

    lora_results["model"].append(model_name)
    lora_results["dataset"].append(dataset_name)
    lora_results["accuracy"].append(acc)
    lora_results["config"].append(str(cfg))

    print("model: {}, dataset: {}, accuracy: {}".format(model_name, dataset_name, acc))

lora_results = pd.DataFrame(lora_results)
lora_results

[18:56:24] INFO     =======load pretrained model and tokenizer========                              ]8;id=205463;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=684865;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

[18:56:27] INFO     set peft seed to 42                                                          ]8;id=472873;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py\finetune_lm.py]8;;\:]8;id=85561;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py#121\121]8;;\

Global seed set to 42


trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3560841867092814


/tmp/ipykernel_1757684/993648505.py:31: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  log.warn(


...

           INFO     load checkpoint from epoch=5_step=2000.pth                                      ]8;id=709570;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=776646;file:///tmp/ipykernel_1757684/993648505.py#35\35]8;;\

           INFO     ==================================================                              ]8;id=442417;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=33326;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
100%|██████████| 47/47 [00:28<00:00,  1.65it/s]

model: flan-t5-base, dataset: glue-stsb, accuracy: 0.14466666666666667


,model,dataset,accuracy,config
0,flan-t5-base,glue-cola,0.690316,{'model': {'model': {'_target_': 'transformers...
1,flan-t5-base,glue-mnli,0.668670,{'model': {'model': {'_target_': 'transformers...
2,flan-t5-base,glue-mrpc,0.700980,{'model': {'model': {'_target_': 'transformers...
3,flan-t5-base,glue-qqp,0.735320,{'model': {'model': {'_target_': 'transformers...
4,flan-t5-base,glue-rte,0.805054,{'model': {'model': {'_target_': 'transformers...
5,flan-t5-base,glue-sst2,0.919725,{'model': {'model': {'_target_': 'transformers...
6,flan-t5-base,glue-stsb,0.144667,{'model': {'model': {'_target_': 'transformers...


In [15]:
os.makedirs(f"results/{model_name}", exist_ok=True)
lora_results.to_csv(f"results/{model_name}/lora_results.csv")

## V1

In [21]:
lora_results = {"model": [], "dataset": [], "accuracy": [], "config": []}

model_name = "flan-t5-base"
finetune_mode = "lora"
for dataset_name in [
    "glue-cola",
    "glue-mnli",
    "glue-mrpc",
    "glue-qqp",
    "glue-rte",
    "glue-sst2",
    "glue-stsb",
]:
    model = load_finetuned_model(model_name, dataset_name, finetune_mode, 1)
    cfg, model, tokenizer = model["config"], model["model"], model["tokenizer"]

    if dataset_name in validataion_dataloaders:
        val_loader = validataion_dataloaders[dataset_name]
    else:
        val_loader = load_validation_dataloaer(cfg)
        validataion_dataloaders[dataset_name] = val_loader

    model = fabric.setup_module(model)
    val_loader = fabric.setup_dataloaders(val_loader)
    acc = evaluate_accuracy(model, val_loader, tokenizer)

    lora_results["model"].append(model_name)
    lora_results["dataset"].append(dataset_name)
    lora_results["accuracy"].append(acc)
    lora_results["config"].append(str(cfg))

    print("model: {}, dataset: {}, accuracy: {}".format(model_name, dataset_name, acc))

lora_results = pd.DataFrame(lora_results)
lora_results

[21:15:13] INFO     =======load pretrained model and tokenizer========                              ]8;id=529903;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=631262;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

[21:15:16] INFO     set peft seed to 42                                                          ]8;id=681453;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py\finetune_lm.py]8;;\:]8;id=735392;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py#121\121]8;;\

Global seed set to 42


trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3560841867092814


/tmp/ipykernel_1757684/993648505.py:31: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  log.warn(


...

           INFO     load checkpoint from epoch=5_step=2000.pth                                      ]8;id=709570;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=776646;file:///tmp/ipykernel_1757684/993648505.py#35\35]8;;\

[21:35:56] INFO     ==================================================                              ]8;id=442417;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=33326;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
100%|██████████| 47/47 [00:28<00:00,  1.65it/s]

model: flan-t5-base, dataset: glue-stsb, accuracy: 0.14933333333333335


,model,dataset,accuracy,config
0,flan-t5-base,glue-cola,0.692234,{'model': {'model': {'_target_': 'transformers...
1,flan-t5-base,glue-mnli,0.717575,{'model': {'model': {'_target_': 'transformers...
2,flan-t5-base,glue-mrpc,0.762255,{'model': {'model': {'_target_': 'transformers...
3,flan-t5-base,glue-qqp,0.792407,{'model': {'model': {'_target_': 'transformers...
4,flan-t5-base,glue-rte,0.801444,{'model': {'model': {'_target_': 'transformers...
5,flan-t5-base,glue-sst2,0.923165,{'model': {'model': {'_target_': 'transformers...
6,flan-t5-base,glue-stsb,0.149333,{'model': {'model': {'_target_': 'transformers...


In [24]:
os.makedirs(f"results/{model_name}", exist_ok=True)
lora_results.to_csv(f"results/{model_name}/lora_results_v1.csv")

## V2-9

In [10]:
for version in range(2, 10):
    lora_results = {"model": [], "dataset": [], "accuracy": [], "config": []}

    model_name = "flan-t5-base"
    finetune_mode = "lora"
    for dataset_name in [
        "glue-cola",
        "glue-mnli",
        "glue-mrpc",
        "glue-qqp",
        "glue-rte",
        "glue-sst2",
        "glue-stsb",
    ]:
        model = load_finetuned_model(model_name, dataset_name, finetune_mode, version)
        cfg, model, tokenizer = model["config"], model["model"], model["tokenizer"]

        if dataset_name in validataion_dataloaders:
            val_loader = validataion_dataloaders[dataset_name]
        else:
            val_loader = load_validation_dataloaer(cfg)
            validataion_dataloaders[dataset_name] = val_loader

        model = fabric.setup_module(model)
        val_loader = fabric.setup_dataloaders(val_loader)
        acc = evaluate_accuracy(model, val_loader, tokenizer)

        lora_results["model"].append(model_name)
        lora_results["dataset"].append(dataset_name)
        lora_results["accuracy"].append(acc)
        lora_results["config"].append(str(cfg))

        print("model: {}, dataset: {}, accuracy: {}".format(model_name, dataset_name, acc))

    lora_results = pd.DataFrame(lora_results)
    lora_results

    os.makedirs(f"results/{model_name}", exist_ok=True)
    lora_results.to_csv(f"results/{model_name}/lora_results_v{version}.csv")

[16:10:07] INFO     =======load pretrained model and tokenizer========                              ]8;id=375986;file:///tmp/ipykernel_379798/993648505.py\993648505.py]8;;\:]8;id=317788;file:///tmp/ipykernel_379798/993648505.py#14\14]8;;\

[16:10:10] INFO     set peft seed to 42                                                          ]8;id=485662;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py\finetune_lm.py]8;;\:]8;id=678469;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py#121\121]8;;\

Global seed set to 42


trainable params: 884,736 || all params: 248,462,592 || trainable%: 0.3560841867092814


/tmp/ipykernel_379798/993648505.py:31: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  log.warn(


...

[19:12:31] WARNING  Using the latest cached version of the module from                                 ]8;id=529903;file:///data/users/tanganke/anaconda3/lib/python3.10/site-packages/datasets/load.py\load.py]8;;\:]8;id=631262;file:///data/users/tanganke/anaconda3/lib/python3.10/site-packages/datasets/load.py#1252\1252]8;;\
                    /data/users/tanganke/.cache/huggingface/modules/datasets_modules/datasets/glue/dac             
                    be3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad (last modified on                
                    Tue Aug  1 12:12:31 2023) since it couldn't be found locally at glue., or remotely             
                    on the Hugging Face Hub.                                                                       

Running tokenizer on dataset:   0%|          | 0/277 [00:00<?, ? examples/s]

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
100%|██████████| 9/9 [00:07<00:00,  1.27it/s]


model: flan-t5-base, dataset: glue-rte, accuracy: 0.8050541516245487


FileNotFoundError: [Errno 2] No such file or directory: '/data/users/tanganke/projects/task_arithmetic/peta/logs/flan-t5-base/glue-sst2/lora/version_2/config.yaml'

# L_LoRA

## V0

In [25]:
# version 0
l_lora_results = {"model": [], "dataset": [], "accuracy": [], "config": []}

model_name = "flan-t5-base"
finetune_mode = "l_lora"
for dataset_name in [
    "glue-cola",
    "glue-mnli",
    "glue-mrpc",
    "glue-qqp",
    "glue-rte",
    "glue-sst2",
    "glue-stsb",
]:
    model = load_finetuned_model(model_name, dataset_name, finetune_mode, 0)
    cfg, model, tokenizer = model["config"], model["model"], model["tokenizer"]

    if dataset_name in validataion_dataloaders:
        val_loader = validataion_dataloaders[dataset_name]
    else:
        val_loader = load_validation_dataloaer(cfg)
        validataion_dataloaders[dataset_name] = val_loader

    model = fabric.setup_module(model)
    val_loader = fabric.setup_dataloaders(val_loader)
    acc = evaluate_accuracy(model, val_loader, tokenizer)

    l_lora_results["model"].append(model_name)
    l_lora_results["dataset"].append(dataset_name)
    l_lora_results["accuracy"].append(acc)
    l_lora_results["config"].append(cfg)

    print("model: {}, dataset: {}, accuracy: {}".format(model_name, dataset_name, acc))

l_lora_results = pd.DataFrame(l_lora_results)
l_lora_results

[22:29:48] INFO     =======load pretrained model and tokenizer========                              ]8;id=638551;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=208573;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

[22:29:51] INFO     set peft seed to 42                                                          ]8;id=565579;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py\finetune_lm.py]8;;\:]8;id=999816;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py#121\121]8;;\

Global seed set to 42


trainable params: 884,736 || all params: 291,814,656 || trainable%: 0.30318422389312755


/tmp/ipykernel_1757684/993648505.py:31: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  log.warn(


...

           INFO     load checkpoint from epoch=5_step=2000.pth                                      ]8;id=709570;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=776646;file:///tmp/ipykernel_1757684/993648505.py#35\35]8;;\

           INFO     ==================================================                              ]8;id=442417;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=33326;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
100%|██████████| 47/47 [00:42<00:00,  1.10it/s]

model: flan-t5-base, dataset: glue-stsb, accuracy: 0.0006666666666666666


,model,dataset,accuracy,config
0,flan-t5-base,glue-cola,0.486098,"[model, peft, dataset, optim, seed, batch_size..."
1,flan-t5-base,glue-mnli,0.000102,"[model, peft, dataset, optim, seed, batch_size..."
2,flan-t5-base,glue-mrpc,0.000000,"[model, peft, dataset, optim, seed, batch_size..."
3,flan-t5-base,glue-qqp,0.000000,"[model, peft, dataset, optim, seed, batch_size..."
4,flan-t5-base,glue-rte,0.000000,"[model, peft, dataset, optim, seed, batch_size..."
5,flan-t5-base,glue-sst2,0.876147,"[model, peft, dataset, optim, seed, batch_size..."
6,flan-t5-base,glue-stsb,0.000667,"[model, peft, dataset, optim, seed, batch_size..."


In [27]:
l_lora_results

,model,dataset,accuracy,config
0,flan-t5-base,glue-cola,0.486098,"[model, peft, dataset, optim, seed, batch_size..."
1,flan-t5-base,glue-mnli,0.000102,"[model, peft, dataset, optim, seed, batch_size..."
2,flan-t5-base,glue-mrpc,0.000000,"[model, peft, dataset, optim, seed, batch_size..."
3,flan-t5-base,glue-qqp,0.000000,"[model, peft, dataset, optim, seed, batch_size..."
4,flan-t5-base,glue-rte,0.000000,"[model, peft, dataset, optim, seed, batch_size..."
5,flan-t5-base,glue-sst2,0.876147,"[model, peft, dataset, optim, seed, batch_size..."
6,flan-t5-base,glue-stsb,0.000667,"[model, peft, dataset, optim, seed, batch_size..."


In [26]:
os.makedirs(f"results/{model_name}", exist_ok=True)
l_lora_results.to_csv(f"results/{model_name}/l_lora_results.csv")

## V1

In [18]:
# version 1
l_lora_results = {"model": [], "dataset": [], "accuracy": [], "config": []}

model_name = "flan-t5-base"
finetune_mode = "l_lora"
for dataset_name in [
    "glue-cola",
    "glue-mnli",
    "glue-mrpc",
    "glue-qqp",
    "glue-rte",
    "glue-sst2",
    "glue-stsb",
]:
    model = load_finetuned_model(model_name, dataset_name, finetune_mode, 1) # version 1
    cfg, model, tokenizer = model["config"], model["model"], model["tokenizer"]

    if dataset_name in validataion_dataloaders:
        val_loader = validataion_dataloaders[dataset_name]
    else:
        val_loader = load_validation_dataloaer(cfg)
        validataion_dataloaders[dataset_name] = val_loader

    model = fabric.setup_module(model)
    val_loader = fabric.setup_dataloaders(val_loader)
    acc = evaluate_accuracy(model, val_loader, tokenizer)

    l_lora_results["model"].append(model_name)
    l_lora_results["dataset"].append(dataset_name)
    l_lora_results["accuracy"].append(acc)
    l_lora_results["config"].append(cfg)

    print("model: {}, dataset: {}, accuracy: {}".format(model_name, dataset_name, acc))

l_lora_results = pd.DataFrame(l_lora_results)
l_lora_results

[20:41:51] INFO     =======load pretrained model and tokenizer========                              ]8;id=529903;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=631262;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

[20:41:54] INFO     set peft seed to 42                                                          ]8;id=681453;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py\finetune_lm.py]8;;\:]8;id=735392;file:///data/users/tanganke/projects/task_arithmetic/peta/finetune_lm.py#121\121]8;;\

Global seed set to 42


trainable params: 884,736 || all params: 291,814,656 || trainable%: 0.30318422389312755


/tmp/ipykernel_1757684/993648505.py:31: DeprecationWarning: The 'warn' method is deprecated, use 'warning' instead
  log.warn(


...

           INFO     load checkpoint from epoch=5_step=2000.pth                                      ]8;id=709570;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=776646;file:///tmp/ipykernel_1757684/993648505.py#35\35]8;;\

           INFO     ==================================================                              ]8;id=442417;file:///tmp/ipykernel_1757684/993648505.py\993648505.py]8;;\:]8;id=33326;file:///tmp/ipykernel_1757684/993648505.py#14\14]8;;\

You are using a CUDA device ('NVIDIA GeForce RTX 3090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
100%|██████████| 47/47 [00:40<00:00,  1.17it/s]

model: flan-t5-base, dataset: glue-stsb, accuracy: 0.03


,model,dataset,accuracy,config
0,flan-t5-base,glue-cola,0.688399,"[model, peft, dataset, optim, seed, batch_size..."
1,flan-t5-base,glue-mnli,0.288640,"[model, peft, dataset, optim, seed, batch_size..."
2,flan-t5-base,glue-mrpc,0.683824,"[model, peft, dataset, optim, seed, batch_size..."
3,flan-t5-base,glue-qqp,0.471407,"[model, peft, dataset, optim, seed, batch_size..."
4,flan-t5-base,glue-rte,0.472924,"[model, peft, dataset, optim, seed, batch_size..."
5,flan-t5-base,glue-sst2,0.917431,"[model, peft, dataset, optim, seed, batch_size..."
6,flan-t5-base,glue-stsb,0.030000,"[model, peft, dataset, optim, seed, batch_size..."


In [19]:
os.makedirs(f"results/{model_name}", exist_ok=True)
l_lora_results.to_csv(f"results/{model_name}/l_lora_results_v1.csv")

## V2-11

In [ ]:
for version in range(2, 12):
    l_lora_results = {"model": [], "dataset": [], "accuracy": [], "config": []}

    model_name = "flan-t5-base"
    finetune_mode = "l_lora"
    for dataset_name in [
        "glue-cola",
        "glue-mnli",
        "glue-mrpc",
        "glue-qqp",
        "glue-rte",
        "glue-sst2",
        "glue-stsb",
    ]:
        model = load_finetuned_model(
            model_name, dataset_name, finetune_mode, version
        )  # version 1
        cfg, model, tokenizer = model["config"], model["model"], model["tokenizer"]

        if dataset_name in validataion_dataloaders:
            val_loader = validataion_dataloaders[dataset_name]
        else:
            val_loader = load_validation_dataloaer(cfg)
            validataion_dataloaders[dataset_name] = val_loader

        model = fabric.setup_module(model)
        val_loader = fabric.setup_dataloaders(val_loader)
        acc = evaluate_accuracy(model, val_loader, tokenizer)

        l_lora_results["model"].append(model_name)
        l_lora_results["dataset"].append(dataset_name)
        l_lora_results["accuracy"].append(acc)
        l_lora_results["config"].append(cfg)

        print(
            "model: {}, dataset: {}, accuracy: {}".format(model_name, dataset_name, acc)
        )

    l_lora_results = pd.DataFrame(l_lora_results)
    l_lora_results

    os.makedirs(f"results/{model_name}", exist_ok=True)
    l_lora_results.to_csv(f"results/{model_name}/l_lora_results_v{version}.csv")